In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse  as sp
from scipy.sparse.linalg import eigsh

#Location of the data_folder 
data_folder      ="/nvme/drive_1/NTDS_Final/"
num_eigenvectors = 100

** I. Calculate Laplacian ** 

In [ ]:
adj                  = sp.load_npz(data_folder+"new_filtering/adjacency_5.npz")
n_nodes              = adj.shape[0]


degrees                  = adj.sum(0)
degree_matrix            = sp.spdiags(degrees,[0],n_nodes,n_nodes)
degree_matrix            = sp.csr_matrix(degree_matrix,degree_matrix.shape,dtype=np.float16)
laplacian_combinatorial  = degree_matrix - adj
newD                     = sp.spdiags(1/np.sqrt(degrees),[0],degrees.size,degrees.size)
normalized_laplacian     = ((newD @ laplacian_combinatorial) @ newD)

sp.save_npz(data_folder+"new_filtering/laplacian.npz",normalized_laplacian)

** II. Calculate Eigen Decomposition ** 

In [ ]:
normalized_laplacian = sp.load_npz(data_folder+"new_filtering/laplacian.npz")
values, vectors      = eigsh(normalized_laplacian, k=num_eigenvectors, which='LM')


In [ ]:
np.save(data_folder+"new_filtering/e_vectors.npy",vectors)
np.save(data_folder+"new_filtering/e_values.npy",values)

** III. Project Laplacian Column onto EigenVectors for Features  ** 

In [ ]:
normalized_laplacian   = sp.load_npz(data_folder+"new_filtering/laplacian.npz")
adj                  = sp.load_npz(data_folder+"new_filtering/adjacency_5.npz")
vectors = np.load(data_folder+"new_filtering/e_vectors.npy")
values  = np.load(data_folder+"new_filtering/e_values.npy")

# Vectors.T is (100 * N), Laplacian is (N * N)
features = vectors.T @ adj
np.save(data_folder+"new_filtering/features.npy",features.T)